# Credit Card Notebook_XGBoost

In [ ]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt

from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier, plot_importance, DMatrix, plot_tree

In [ ]:
% cd /Users/MiroWang/Desktop/Github Repo/CreditFraudData

In [ ]:
train_frame = pd.read_csv("train_frame_creditcard.csv", index_col = 0)

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
X_xgb = train_frame[train_frame.columns[:-1]].as_matrix()
y_xgb = train_frame['Class']

strsplit = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=36)
for train_index, test_index in strsplit.split(X_rf, y_rf):
    X_train_rf, X_test_rf = X_rf[train_index], X_rf[test_index]
    y_train_rf, y_test_rf = y_rf[train_index], y_rf[test_index]
    
print(X_train_rf.shape, X_test_rf.shape, y_train_rf.shape, y_test_rf.shape)